In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
cwd = '../data/raw_data/'
clean_dir = '../data/clean_data/'


mobility = pd.read_csv(cwd+'2020_US_Region_Mobility_Report.csv') #read in data
mobility.head()
max(mobility['date'])

FileNotFoundError: [Errno 2] File ../data/raw_data/2020_US_Region_Mobility_Report.csv does not exist: '../data/raw_data/2020_US_Region_Mobility_Report.csv'

In [9]:
mobility['date'] = pd.to_datetime(mobility['date']) #convert str object to datetime
date_after = pd.to_datetime('2020-03-20') #this is where our twitter data begins
date_before = pd.to_datetime('2020-12-01') #this is where our twitter data ends
mobility = mobility[mobility['date'] >= date_after]
mobility = mobility[mobility['date'] <= date_before]
mobility = mobility.rename(columns={"retail_and_recreation_percent_change_from_baseline": "retail_rec", 
                                    "grocery_and_pharmacy_percent_change_from_baseline": "groc_pharm",
                                   "parks_percent_change_from_baseline": "parks",
                                   "transit_stations_percent_change_from_baseline": "transit",
                                   "workplaces_percent_change_from_baseline": "workplaces",
                                   "residential_percent_change_from_baseline": "residential"})
mobility.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_rec,groc_pharm,parks,transit,workplaces,residential
34,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-20,-32.0,5.0,-2.0,-37.0,-36.0,16.0
35,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-21,-40.0,-2.0,-3.0,-36.0,-24.0,12.0
36,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-22,-44.0,-13.0,-12.0,-44.0,-31.0,11.0
37,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-23,-38.0,-10.0,-23.0,-47.0,-41.0,17.0
38,US,United States,NaN,NaN,NaN,NaN,NaN,2020-03-24,-39.0,-12.0,-10.0,-46.0,-45.0,19.0


In [10]:
fl = mobility[mobility['sub_region_1'] == 'Florida']
fl = fl[fl['sub_region_2'].notna()]

In [11]:
fl.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_rec,groc_pharm,parks,transit,workplaces,residential
70287,US,United States,Florida,Alachua County,NaN,NaN,12001.0,2020-03-20,-30.0,3.0,-17.0,-29.0,-33.0,16.0
70288,US,United States,Florida,Alachua County,NaN,NaN,12001.0,2020-03-21,-42.0,-2.0,-31.0,-26.0,-20.0,10.0
70289,US,United States,Florida,Alachua County,NaN,NaN,12001.0,2020-03-22,-45.0,-11.0,-18.0,-35.0,-29.0,10.0
70290,US,United States,Florida,Alachua County,NaN,NaN,12001.0,2020-03-23,-36.0,-3.0,-21.0,-35.0,-41.0,16.0
70291,US,United States,Florida,Alachua County,NaN,NaN,12001.0,2020-03-24,-46.0,-20.0,-29.0,-40.0,-46.0,20.0


In [12]:
fl.to_csv(clean_dir+'florida_mobility.csv') #read in data